In [1]:
import requests 
from requests import get
from bs4 import BeautifulSoup
import time

from journals_lista import lista_journals

In [2]:
# Lista de listas
# cada lista interna son los urls de los journals escogidos

urls_jounrlas = lista_journals()

# lista de cada journal con su codigo
urlss = ["aea/aecrev","ecm/emetrp","wly/emetrp","oup/qjecon","ucp/jpolec","oup/restud"]

In [3]:
# FUNCION QUE REESCRIBE Y ORDENA LOS NOMBRES DE LOS AUTORES DE UN PAPER
# SE AUTOIMPLEMENTA MAS ADELANTE
# IMPLEMENTACION DENTRO DE LA FUNCION writeLine(dirr)

def nomu(long):
    if '"' in long[0]:
        long[0] = long[0][1:]
    if '"' in long[-1]:
        long[-1] = long[-1][:-2]
    b = long[1:-1]
    c = [i.split(" & ") for i in b]
    d = [";".join(i) for i in c]
    e = "".join(d)
    f = "".join([long[0],e,long[-1]])
    g = f.split(";")
    return g

In [4]:
# FUNCION QUE TOMA UN URL (EN ESTE CASO DE LA LISTA DE URLS DE PAPERS)
# Y DEVUELVE UNA LISTA CON TODAS LAS LINEAS QUE SE ESCRIBIRAN EN EL .cvs

def writeLine(dirr):
    response = get(dirr)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    datas = []
    for link in html_soup.find_all("i"):
        repec = link.text
        if len(repec)>20:
            if repec[-1]=='.':
                repec = repec[:-1]
            parti = repec.split(":")
            year = parti[-5]
            vol = parti[4]
            issue = parti[-3]
            pages = parti[-1].split("-")
            pag0 = pages[0]
            pag1 = pages[1]
            data = [repec, year, vol, issue, pag0, pag1]
            if repec not in datas:
                datas+=data
    texto = ""
    for link in html_soup.find_all("script"):
        cadena = "Inner Text: {}".format(link.text)
        if "schema.org" in cadena:
            texto+=cadena
    part1 = texto.split('name": "')
    part2 = part1[1:]
    datos = []
    for i in part2:
        if "issueNumber" in i:
            part3 = i.split('",')
            journal = part3[0]
            datos.append(journal)
        elif "pagination" in i:
            part4 = i.split('",')
            title = part4[0]
            autores = i.split('author": "')[1].split('",')[0]
            nombres = nomu(autores)
            datos.append(title)
            datos.append(nombres)
    L = []
    authrs = datos[2]
    for name in authrs:
        general = [datas[0]] + datos[:2][::-1] + datas[1:] + [name]
        linea = "|".join(general) + '\n'
        L.append(linea)
    return L

In [ ]:
# se procede a crear un archivo por cada lista
# cada archivo creado correspondera a cada journal
# el proceso recorrera journal by journal

for i in range(len(urlss)):
    jrnl = urlss[i].split("/")[-1] # abreviatura del journal
    file = open("{}.csv".format(jrnl), "a+") # creacion del archivo con la abreviatura del journal
    
    file.write('handle,title,journal,year,vol,issue,page,end page,author\n') # se escribe el formato inicial
    X = []
    print(time.ctime())
    
    # a partir de aqui se procesa cada lista con sus respectivos urls
    # se deja correr el codigo
    # se recomienda servirse un cafe cargado
    
    for dirr in urls_jounrlas[i]:
        time.ctime()
        try:
            W = writeLine(dirr)
            for w in W:
                if w not in X:
                    print(w)
                    file.write(w)
                    X.append(w)
        except (UnicodeEncodeError, IndexError):
            print("No es str")
        except:
            print("Error code review")
        time.ctime()
    file.close()

Sun Jan 26 13:24:20 2020
RePEc:aea:aecrev:v:100:y:2010:i:1:p:5-34|Price Indexes, Inequality, and the Measurement of World Poverty|American Economic Review|2010|100|1|5|34|Angus Deaton

RePEc:aea:aecrev:v:100:y:2010:i:1:p:35-69|Learning about a New Technology: Pineapple in Ghana|American Economic Review|2010|100|1|35|69|Timothy G. Conley & Christopher R. Udry

RePEc:aea:aecrev:v:100:y:2010:i:1:p:70-97|Multiple-Product Firms and Product Switching|American Economic Review|2010|100|1|70|97|Andrew B. Bernard & Stephen J. Redding & Peter K. Schott

RePEc:aea:aecrev:v:100:y:2010:i:1:p:98-129|&quot;Momma's Got the Pill&quot;: How Anthony Comstock and Griswold v. Connecticut Shaped US Childbearing|American Economic Review|2010|100|1|98|129|Martha J. Bailey

RePEc:aea:aecrev:v:100:y:2010:i:1:p:130-63|Matching and Sorting in Online Dating|American Economic Review|2010|100|1|130|63|Gunter J. Hitsch & Ali Hortaçsu & Dan Ariely

RePEc:aea:aecrev:v:100:y:2010:i:1:p:164-92|Entry, Exit, and Investment-